In [1]:
import torch
import malt

dummy_targets = torch.Tensor([0.0])

regressor = malt.models.regressor.GPyTorchExactRegressor(
    dummy_targets,
    in_features=128,
    out_features=2,
)

Using backend: pytorch


In [2]:
import torch
import dgl
import malt

dummy_targets = torch.Tensor([0.0])

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.GPyTorchExactRegressor(
        dummy_targets,
        in_features=128,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

point = malt.Molecule("C")
point.featurize()
graph = dgl.batch([point.g])

y = net.condition(graph)
assert y.mean.item() == 0.0

In [3]:
import torch
import dgl
import malt
import gpytorch

dummy_targets = torch.tensor([[4.0]])

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=32
    ),
    regressor=malt.models.regressor.GPyTorchExactRegressor(
        dummy_targets,
        in_features=32,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

point = malt.Molecule("CCCCCCC")
point.featurize()
graph = dgl.batch([point.g])

mll = gpytorch.mlls.ExactMarginalLogLikelihood(net.regressor.likelihood, net)

net.train()
y_pred = net(graph)
loss = mll(y_pred, torch.tensor([[2.0]]))
loss.backward()

net.eval()
y = net(graph)
assert y.mean.item() != 0.0

In [5]:
import torch
import dgl
import malt

dataset = malt.data.collections.linear_alkanes(10)
dataset_loader = dataset.view(batch_size=len(dataset))
g, y = next(iter(dataset_loader))

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.GPyTorchExactRegressor(
        y,
        in_features=128,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

if torch.cuda.is_available():
    net.cuda()

mll = gpytorch.mlls.ExactMarginalLogLikelihood(net.regressor.likelihood, net)

net.train()
y_pred = net(g)
loss = mll(y_pred, y).mean()
loss.backward()

net.eval()
y_hat = net(g)
assert y_hat.mean.shape[0] == 10
assert len(y_hat.mean.shape) == 1

In [13]:
import malt
import torch
from malt.agents.player import SequentialModelBasedPlayer

dataset = malt.data.collections.linear_alkanes(10)
g, y = dataset.batch()
model = malt.models.supervised_model.GaussianProcessSupervisedModel(
   representation=malt.models.representation.DGLRepresentation(
       out_features=32
   ),
   regressor=malt.models.regressor.ExactGaussianProcessRegressor(
       y, in_features=32, out_features=2,
   ),
   likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)
if torch.cuda.is_available():
    model.cuda()
    
mll = malt.models.marginal_likelihood.ExactMarginalLogLikelihood(
    model.regressor.likelihood,
    model
)

player = SequentialModelBasedPlayer(
    model = model,
    policy=malt.policy.Greedy(),
    trainer=malt.trainer.get_default_trainer(),
    marginal_likelihood=mll,
    merchant=malt.agents.merchant.DatasetMerchant(dataset),
    assayer=malt.agents.assayer.DatasetAssayer(dataset),
)

while True:
    if player.step() is None:
        break

In [3]:
import torch
import dgl
import malt

dataset = malt.data.collections.linear_alkanes(10)
dataset_loader = dataset.view(batch_size=len(dataset))
g, y = next(iter(dataset_loader))


net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.HardcodedExactGaussianProcessRegressor(
        y,
        in_features=128,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

if torch.cuda.is_available():
    net.cuda()


mll = malt.models.marginal_likelihood.HardcodedExactMarginalLogLikelihood(
    net.likelihood, net
)

y_hat = net(g)
loss = mll(y_hat, y)
loss.backward()

y_hat = net.condition(g)
assert y_hat.mean.shape[0] == 10
assert len(y_hat.mean.shape) == 1

In [1]:
import torch
import dgl
import malt

dummy_targets = torch.Tensor([4.0])

net = malt.models.supervised_model.GaussianProcessSupervisedModel(
    representation=malt.models.representation.DGLRepresentation(
        out_features=128
    ),
    regressor=malt.models.regressor.HardcodedExactGaussianProcessRegressor(
        dummy_targets,
        in_features=128,
        out_features=2,
    ),
    likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)

point = malt.Molecule("C")
point.featurize()
graph = dgl.batch([point.g])

mll = malt.models.marginal_likelihood.HardcodedExactMarginalLogLikelihood(
    net.likelihood, net
)

net.train()
y_hat = net(graph)
loss = mll(y_hat, torch.tensor([[5.0]]))
loss.backward()

net.eval()
y = net.condition(graph)
assert y.mean.item() != 0.0

Using backend: pytorch


In [12]:
%load_ext autoreload
%autoreload 2

import malt
import torch
from malt.agents.player import SequentialModelBasedPlayer

dataset = malt.data.collections.linear_alkanes(10)
g, y = dataset.batch()
model = malt.models.supervised_model.GaussianProcessSupervisedModel(
   representation=malt.models.representation.DGLRepresentation(
       out_features=32
   ),
   regressor=malt.models.regressor.ExactGaussianProcessRegressor(
       y, in_features=32, out_features=2,
   ),
   likelihood=malt.models.likelihood.HeteroschedasticGaussianLikelihood(),
)
if torch.cuda.is_available():
    model.cuda()

mll = malt.models.marginal_likelihood.ExactMarginalLogLikelihood(
    model.regressor.likelihood,
    model
)

player = SequentialModelBasedPlayer(
    model = model,
    policy=malt.policy.Greedy(),
    trainer=malt.trainer.get_default_trainer(),
    marginal_likelihood=mll,
    merchant=malt.agents.merchant.DatasetMerchant(dataset),
    assayer=malt.agents.assayer.DatasetAssayer(dataset),
)

while True:
    if player.step() is None:
        break

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
g, y = ds.batch(by=['g', 'y'])
torch.nn.functional.mse_loss(DumbModel(input_)(g).mean, target)

tensor(1.2900)

In [ ]:
class DumbModel(object):
    def __init__(self, y):
        self.y = y

    def __call__(self, *args, **kwargs):
        return torch.distributions.Normal(
            self.y.ravel().cuda(), 1.0
        )

import torch
import malt
from malt.metrics.supervised_metrics import MSE, MAPE, RMSE, R2
from malt.metrics.base_metrics import mse, mape, rmse, r2
input_ = torch.randn(5, 1)
target = torch.randn(5, 1)

ds = malt.Dataset()

for idx in range(5):
    point = malt.Molecule(smiles="C", metadata={'y': target[idx].item()})
    ds.append(point)

assert MSE()(DumbModel(input_), ds) == mse(input_, target)
assert MAPE()(DumbModel(input_), ds) == mape(input_, target)
assert RMSE()(DumbModel(input_), ds) == rmse(input_, target)
assert R2()(DumbModel(input_), ds) == r2(input_, target)

10


c:\users\micha\dev\choderalab\malt\malt\metrics\base_metrics.py:4: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return torch.nn.functional.mse_loss(target, input)


In [42]:
g, y = ds.batch(by=['g', 'y'])
y_hat = DumbModel(input_)(g).mean
malt.metrics.base_metrics.mse(y_hat, target)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [12]:
MSE()(DumbModel(input_), ds)

tensor(1.8533, device='cuda:0')

In [5]:
mse(input_, target)

tensor(1.6798)